[Sascha Spors](https://orcid.org/0000-0001-7225-9992),
Professorship Signal Theory and Digital Signal Processing,
[Institute of Communications Engineering (INT)](https://www.int.uni-rostock.de/),
Faculty of Computer Science and Electrical Engineering (IEF),
[University of Rostock, Germany](https://www.uni-rostock.de/en/)

# Tutorial Signals and Systems (Signal- und Systemtheorie)

Summer Semester 2023 (Bachelor Course #24015)

- lecture: https://github.com/spatialaudio/signals-and-systems-lecture
- tutorial: https://github.com/spatialaudio/signals-and-systems-exercises

Feel free to contact lecturer [frank.schultz@uni-rostock.de](https://orcid.org/0000-0002-3010-0294)

## Übung / Exercise 5 Bode Plot of LTI Systems

### References

* Norbert Fliege (1991): "*Systemtheorie*", Teubner, Stuttgart (GER), cf. chapter 4.3.5

* Alan V. Oppenheim, Alan S. Willsky with S. Hamid Nawab (1997): "*Signals & Systems*", Prentice Hall, Upper Saddle River NJ (USA), 2nd ed., cf. chapter 6

* Bernd Girod, Rudolf Rabenstein, Alexander Stenger (2001): "*Signals and Systems*", Wiley, Chichester (UK), cf. chapter 10

* Bernd Girod, Rudolf Rabenstein, Alexander Stenger (2005/2007): "*Einführung in die Systemtheorie*", Teubner, Wiesbaden (GER), 3rd/4th ed., cf. chapter 10

Let's import required packages and define some helping routines for plotting in the following.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import scipy.signal as signal

import os
import sys
cur_fol = os.getcwd()
print(cur_fol)
sys.path.append(cur_fol + '/../')

# %matplotlib inline

from sig_sys_tools import plot_clti_analysis

In [ ]:
def lti_bode_plot(sys, txt):
    w, mag, phase = sys.bode(np.logspace(-2, 2, 2**8))

    fig = plt.figure(figsize=(6, 5), tight_layout=True)
    plt.subplot(2, 1, 1)
    plt.title(txt)
    plt.semilogx(w, mag, 'C0', linewidth=3)
    plt.grid(True)
    plt.xlabel('$\omega$ / (rad/s)')
    plt.ylabel('Level in dB')
    plt.xlim(w[0], w[-1])

    plt.subplot(2, 1, 2)
    plt.semilogx(w, phase, 'C0', linewidth=3)
    plt.grid(True)
    plt.xlabel('$\omega$ / (rad/s)')
    plt.ylabel('Phase in deg')
    plt.xlim(w[0], w[-1])

In [ ]:
# plot LTI system characteristics for Laplace domain
def plot_LTIs(sys):
    z = np.squeeze(sys.zeros)
    p = np.squeeze(sys.poles)
    k = sys.gain

    w = np.logspace(-2, 2, 2**8)
    w, H = signal.freqs_zpk(z, p, k, w)
    th, h = signal.impulse(sys)
    the, he = signal.step(sys)

    plt.figure(figsize=(7, 9), tight_layout=True)

    # pole / zero plot
    plt.subplot(325)

    # clear poles / zeros that compensate each other, TBD: numerical robust,
    # works for didactical purpose
    sz_tmp = z
    sp_tmp = p
    zp = np.array([1])
    while zp.size != 0:
        zp, z_ind, p_ind = np.intersect1d(sz_tmp, sp_tmp, return_indices=True)
        sz_tmp = np.delete(sz_tmp, z_ind)
        sp_tmp = np.delete(sp_tmp, p_ind)
    z = sz_tmp
    p = sp_tmp

    # hard coded ymin, width, height -> TBD
    # works for the didactical examples below
    rect_box = Rectangle((np.max(p.real), -99), 2*99, 2*99,
                         color='yellowgreen', alpha=0.33)
    plt.gcf().gca().add_artist(rect_box)

    zu, zc = np.unique(z, return_counts=True)  # find and count unique zeros
    for zui, zci in zip(zu, zc):  # plot them individually
        plt.plot(np.real(zui), np.imag(zui), ms=10,
                 color='C0', marker='o', fillstyle='none')
        if zci > 1:  # if multiple zeros exist then indicate the count
            plt.text(np.real(zui), np.imag(zui), zci, color='C0',
                     fontsize=14, fontweight='bold')

    pu, pc = np.unique(p, return_counts=True)  # find and count unique poles
    for pui, pci in zip(pu, pc):  # plot them individually
        plt.plot(np.real(pui), np.imag(pui), ms=10,
                 color='C3', marker='x')
        if pci > 1:  # if multiple poles exist then indicate the count
            plt.text(np.real(pui), np.imag(pui), pci, color='C3',
                     fontsize=14, fontweight='bold')

    plt.axis("equal")
    plt.xlabel(r'$\Re\{s\}$')
    plt.ylabel(r'$\Im\{s\}$')
    plt.grid(True)
    plt.title("Pole/Zero/Gain Map, gain=%f" % k)

    # Nyquist plot
    plt.subplot(326)
    plt.plot(H.real, H.imag, "C0", label="$\omega>0$")
    plt.plot(H.real, -H.imag, "C1", label="$\omega<0$")
    plt.plot(H.real[0], H.imag[0], marker='$w=0$', markersize=25, color="C0")
    plt.axis("equal")
    plt.legend()
    plt.xlabel(r'$\Re\{H(s)\}$')
    plt.ylabel(r'$\Im\{H(s)\}$')
    plt.grid(True)
    plt.title("Nyquist Plot")

    # magnitude response
    plt.subplot(321)
    plt.semilogx(w, 20*np.log10(np.abs(H)))
    plt.xlabel('$\omega$ / (rad/s)')
    plt.ylabel(r'$A$ / dB')
    plt.grid(True)
    plt.title("Level")
    plt.xlim((w[0], w[-1]))
    #plt.ylim((-60, 6))
    #plt.yticks(np.arange(-60, +12, 6))

    # phase response
    plt.subplot(323)
    plt.semilogx(w, np.unwrap(np.angle(H))*180/np.pi)
    plt.xlabel('$\omega$ / (rad/s)')
    plt.ylabel(r'$\phi$ / deg')
    plt.grid(True)
    plt.title("Phase")
    plt.xlim((w[0], w[-1]))
    plt.ylim((-180, +180))
    plt.yticks(np.arange(-180, +180+45, 45))

    # impulse response
    plt.subplot(322)
    plt.plot(th, h)
    plt.xlabel('t / s')
    plt.ylabel('h(t)')
    plt.grid(True)
    plt.title("Impulse Response")

    # step response
    plt.subplot(324)
    plt.plot(the, he)
    plt.xlabel('t / s')
    plt.ylabel('h$_\epsilon$(t)')
    plt.grid(True)
    plt.title("Step Response")

# Example: Unity Gain

In [ ]:
sz = []
sp = []
H0 = +1
txt = 'Unity Gain'
sys = signal.lti(sz, sp, H0)
lti_bode_plot(sys, txt)

# Example: Gain and Polarity

In [ ]:
sz = []
sp = []
H0 = -10
txt = '20 dB Gain with Inverted Polarity'
sys = signal.lti(sz, sp, H0)
lti_bode_plot(sys, txt)

# Example: Poles / Zeros in Origin

In [ ]:
sz = 0, 0  # note: more zeros than poles is not a causal system!
sp = 0,
H0 = 1
sys = signal.lti(sz, sp, H0)
txt = str(len(sz)) + ' Zeros / ' + str(len(sp)) + ' Poles in Origin'
txt1 = (': ' + str((len(sz)-len(sp))*20) + ' dB / decade')
lti_bode_plot(sys, txt+txt1)

In [ ]:
sz = 0,
sp = 0, 0, 0
H0 = 1
sys = signal.lti(sz, sp, H0)
txt = str(len(sz)) + ' Zeros / ' + str(len(sp)) + ' Poles in Origin'
txt1 = (': ' + str((len(sz)-len(sp))*20) + ' dB / decade')
lti_bode_plot(sys, txt+txt1)

# Example: Single Real Pole, PT1

In [ ]:
sz = []
sp = -1
H0 = 1
sys = signal.lti(sz, sp, H0)
txt = 'Single Real Pole, decreasing slope, -20 dB / decade'
lti_bode_plot(sys, txt)

# Example: Single Real Zero

In [ ]:
sz = -1
sp = []
H0 = 1
sys = signal.lti(sz, sp, H0)
txt = 'Single Real Zero, increasing slope, + 20 dB / decade'
lti_bode_plot(sys, txt)

# Example: Complex Conjugate Zero Pair

In [ ]:
sz = -3/4-1j, -3/4+1j
sp = []
H0 = 16/25
sys = signal.lti(sz, sp, H0)
txt = 'Conjugate Complex Zero'
lti_bode_plot(sys, txt)

# Examples: Complex Conjugate Pole Pair, PT2

In [ ]:
sz = []
sp = -3/4-1j, -3/4+1j
H0 = 25/16
sys = signal.lti(sz, sp, H0)
txt = 'Conjugate Complex Pole, -3/4$\pm$1j'
lti_bode_plot(sys, txt)

In [ ]:
sz = []
sp = -1/2-1j, -1/2+1j
H0 = 5/4
sys = signal.lti(sz, sp, H0)
txt = 'Conjugate Complex Pole, -1/2$\pm$1j'
lti_bode_plot(sys, txt)

In [ ]:
sz = []
sp = -1/4-1j, -1/4+1j
H0 = 17/16
sys = signal.lti(sz, sp, H0)
txt = 'Conjugate Complex Pole, -1/4$\pm$1j'
lti_bode_plot(sys, txt)

In [ ]:
sz = []
sp = -1/8-1j, -1/8+1j
H0 = 65/64
sys = signal.lti(sz, sp, H0)
txt = 'Conjugate Complex Pole, -1/8$\pm$1j'
lti_bode_plot(sys, txt)

# Example: Integrator, I

In [ ]:
sz = []
sp = 0
H0 = 1
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)
plt.savefig('bodeplot_examples_i_element.pdf')

In [ ]:
plot_clti_analysis(z=np.array(sz), p=np.array([sp]), k=H0);

# Example: Lowpass 1st Order, PT1

In [ ]:
sz = []
sp = -1
H0 = np.abs(sp)
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)
plt.savefig('bodeplot_examples_pt1_element.pdf')

In [ ]:
plot_clti_analysis(z=np.array(sz), p=np.array([sp]), k=H0);

# Example: Highpass 1st Order, DT1

In [ ]:
sz = 0
sp = -1
H0 = 1
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)
plt.savefig('bodeplot_examples_dt1_element.pdf')

In [ ]:
plot_clti_analysis(z=np.array([sz]), p=np.array([sp]), k=H0);

# Example: Allpass 1st Order

In [ ]:
sz = +1
sp = -1
H0 = -1
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)
plt.savefig('bodeplot_examples_ap1_element.pdf')

In [ ]:
plot_clti_analysis(z=np.array([sz]), p=np.array([sp]), k=H0);

# Example: Lowpass 2nd Order, PT2

In [ ]:
sz = []
sp = 1*np.exp(+1j*3*np.pi/4), 1*np.exp(-1j*3*np.pi/4)
H0 = 1
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)
plt.savefig('bodeplot_examples_pt2_element.pdf')

In [ ]:
plot_clti_analysis(z=np.array(sz), p=np.array(sp), k=H0);

# Example: Highpass 2nd Order, DT2

In [ ]:
sz = 0, 0
sp = 1*np.exp(+1j*3*np.pi/4), 1*np.exp(-1j*3*np.pi/4)
H0 = 1
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)
plt.savefig('bodeplot_examples_dt2_element.pdf')

In [ ]:
plot_clti_analysis(z=np.array(sz), p=np.array(sp), k=H0);

# Example: Bandpass 2nd Order

In [ ]:
sz = 0
# sp = 1*np.exp(+1j*2.1*np.pi/4), 1*np.exp(-1j*2.1*np.pi/4)  # high Q
sp = 1*np.exp(+1j*3*np.pi/4), 1*np.exp(-1j*3*np.pi/4)
# sp = 1*np.exp(+1j*3.9*np.pi/4), 1*np.exp(-1j*3.9*np.pi/4)  # low Q
H0 = np.sqrt(2)
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)
plt.savefig('bodeplot_examples_bp2_element.pdf')

In [ ]:
plot_clti_analysis(z=np.array([sz]), p=np.array(sp), k=H0);

# Example: Bandstop 2nd Order

In [ ]:
sz = -1j, +1j
# sp = 1*np.exp(+1j*2.1*np.pi/4), 1*np.exp(-1j*2.1*np.pi/4)  # high Q
sp = 1*np.exp(+1j*3*np.pi/4), 1*np.exp(-1j*3*np.pi/4)
# sp = 1*np.exp(+1j*3.9*np.pi/4), 1*np.exp(-1j*3.9*np.pi/4)  # low Q
H0 = 1
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)
plt.savefig('bodeplot_examples_bs2_element.pdf')

In [ ]:
plot_clti_analysis(z=np.array(sz), p=np.array(sp), k=H0);

# Example: Allpass 2nd Order

In [ ]:
sz = 1*np.exp(+1j*1*np.pi/4), 1*np.exp(-1j*1*np.pi/4)
sp = 1*np.exp(+1j*3*np.pi/4), 1*np.exp(-1j*3*np.pi/4)
H0 = -1
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)
plt.savefig('bodeplot_examples_ap2_element.pdf')

In [ ]:
plot_clti_analysis(z=np.array(sz), p=np.array(sp), k=H0);

# Example: PIT1

In [ ]:
sz = -1
sp = 0, -10
H0 = 1
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)

In [ ]:
plot_clti_analysis(z=np.array([sz]), p=np.array(sp), k=H0);

# Example: PIDT

In [ ]:
sz = -2, -5/2
sp = 0, -10
H0 = 1
sys = signal.lti(sz, sp, H0)
plot_LTIs(sys)

In [ ]:
plot_clti_analysis(z=np.array(sz), p=np.array(sp), k=H0);

# Example: Lowpass 2nd Order, PT2

The 2nd order lopwass
\begin{align}
H_\mathrm{Low}(s) = \frac{1}{\frac{16}{25}s^2+\frac{24}{25}s +1} = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}
\end{align}
is to be characterized by the pole/zero map, the Nyquist plot, the bode plot, the impulse response and the step response.

In [ ]:
w0 = 5/4
D = 3/5
Q = 1/(2*D)
print("D = %4.3f, Q = %4.3f" % (D, Q))

# these are all the same cases:
A = (1/w0**2, 2*D/w0, 1)
print("A = ", A)
A = (1/w0**2, 1/(Q*w0), 1)
print("A = ", A)
A = (16/25, 24/25, 1)
print("A = ", A)

B = (0, 0, 1)

z, p, k = signal.tf2zpk(B, A)
sys = signal.lti(z, p, k)
plot_LTIs(sys)

In [ ]:
plot_clti_analysis(z=z, p=p, k=1);

# Example: Lowpass to Highpass Transform
The Laplace transfer function of a lowpass is transformed to a highpass by exchanging
\begin{align}
s \rightarrow \frac{1}{s}
\end{align}

The 2nd order lopwass $H_\mathrm{Low}(s) = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}$ yields the
following 2nd order highpass filter. It is to be characterized by the pole/zero map, the Nyquist plot, the bode plot, the impulse response and the step response.

In [ ]:
# highpass 2nd order, from lowpass 2nd order with s -> 1/s
A = (1, 24/25, 16/25)
B = (1, 0, 0)
z, p, k = signal.tf2zpk(B, A)
sys = signal.lti(z, p, k)
plot_LTIs(sys)

In [ ]:
plot_clti_analysis(z=z, p=p, k=1);

# Example: Lowpass to Bandpass Transform

The Laplace transfer function of a lowpass is transformed to a bandpass by exchanging 
\begin{align}
s \rightarrow s + \frac{1}{s}
\end{align}
The 2nd order lopwass $H_\mathrm{Low}(s) = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}$ yields to the
following 4th order bandpass filter. It is to be characterized by the pole/zero map, the Nyquist plot, the bode plot, the impulse response and the step response.

In [ ]:
# bandpass 4th order, from lowpass 2nd order with s -> s + 1/s
A = (16, 24, 57, 24, 16)
B = (0, 0, 25, 0, 0)
z, p, k = signal.tf2zpk(B, A)
sys = signal.lti(z, p, k)
plot_LTIs(sys)

In [ ]:
plot_clti_analysis(z=z, p=p, k=1);

# Example: Lowpass to Bandstop Transform

The Laplace transfer function of a lowpass is transformed to a bandstop filter by exchanging
\begin{align}
s \rightarrow \frac{1}{s + \frac{1}{s}}
\end{align}

The 2nd order lopwass $H_\mathrm{Low}(s) = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}$ yields to the
following 4th order bandstop filter. It is to be characterized by the pole/zero map, the Nyquist plot, the bode plot, the impulse response and the step response.

In [ ]:
# bandstop 4th order, from lowpass 2nd order with s -> 1 / (s + 1/s)
A = (25, 24, 66, 24, 25)
B = (25, 0, 50, 0, 25)
z, p, k = signal.tf2zpk(B, A)
sys = signal.lti(z, p, k)
plot_LTIs(sys)

In [ ]:
plot_clti_analysis(z=z, p=p, k=1);

# Example: Bandpass from Real Poles
* zero in origin $s_0=0$
* pole at $s_{\infty,1}=-0.1$
* pole at $s_{\infty,2}=-10$
* $H_0$ = 10

\begin{align}
H(s) = H_0\frac{s-s_{0,1}}{(s-s_{\infty,1})(s-s_{\infty,2})} = 10\frac{(s-0)}{(s+0.1)(s+10)}
=\frac{100 s}{10 s^2 + 101 s + 10}
\end{align}

In [ ]:
if True:
    sz = 0
    sp = -0.1, -10
    H0 = 10
    sys = signal.lti(sz, sp, H0)
else:
    B = (0, 100, 0)
    A = (10, 101, 10)
    sys = signal.lti(B, A)
txt = 'Bandpass'
lti_bode_plot(sys, txt)

# Example: Highpass with Slight Resonance

\begin{align}H(s) = \frac{s^2+10 s}{s^2+\sqrt{2} s +1}\end{align}

In [ ]:
sz = 0, 10
sp = (-1+1j)/np.sqrt(2), (-1-1j)/np.sqrt(2)
H0 = 1
sys = signal.lti(sz, sp, H0)
txt = 'Highpass with Slight Resonance'
lti_bode_plot(sys, txt)

## Copyright

This tutorial is provided as Open Educational Resource (OER), to be found at
https://github.com/spatialaudio/signals-and-systems-exercises
accompanying the OER lecture
https://github.com/spatialaudio/signals-and-systems-lecture.
Both are licensed under a) the Creative Commons Attribution 4.0 International
License for text and graphics and b) the MIT License for source code.
Please attribute material from the tutorial as *Frank Schultz,
Continuous- and Discrete-Time Signals and Systems - A Tutorial Featuring
Computational Examples, University of Rostock* with
``github URL, commit number and/or version tag, year, (file name and/or content)``.